In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
#Einlesen
url = "https://www.berufsberatung.ch/dyn/show/2930"
r = requests.get(url)
r = BeautifulSoup(r.text, "html.parser")

In [3]:
#Job IDs finden
jobs = []
for elem in r.find_all('div', {'class', 'row'})[8].find_all('a')[3:]:
    try:
        mini_dict = {"Jobtitel":elem["data-value"], "JobID": elem["data-value2"]}
        jobs.append(mini_dict)
    except:
        continue

In [4]:
df = pd.DataFrame(jobs)

In [5]:
df.head(50)

,Jobtitel,JobID
0,Abdichter/in EFZ,51914.1
1,Abdichtungspraktiker/in EBA,51919.1
2,Agrarpraktiker/in EBA,15008.2
3,Agrarpraktiker/in (Landwirtschaft) EBA,15009.2
4,Agrarpraktiker/in (Spezialkulturen) EBA,15010.2
5,Agrarpraktiker/in (Weinbereitung) EBA,15011.2
6,Anlagen- und Apparatebauer/in EFZ,44727.1
7,Anlagenführer/in EFZ,44702.1
8,Apparateglasbläser/in EFZ,40105.1
9,Architekturmodellbauer/in EFZ,54603.1


In [6]:
#Mit den Lehrstellen IDs alle Lehrstellen nach Berufen auslesen
url = "https://www.berufsberatung.ch/dyn/show/2930?lang=de&Idx=0&OrderBy=1&Order=0&PostBackOrder=0&postBack=true&CountResult=12&Total_Idx=11&CounterSearch=3&UrlAjaxWebSearch=%2FLenaWeb%2FAjaxWebSearch&prof_=51914.1&LocName=&LocId=&Area=10&schoolyear_=2020&schoolyear_=2021&langcode_=de&langcode_=fr&langcode_=it&langcode_=rm&langcode_=en"

In [7]:
r = requests.get(url)
r = BeautifulSoup(r.text, "html.parser")
list_of_jobs = r.find_all('div', {'class':'border-bottom-0-important'})

In [8]:
#Testen für Firmen
r.find_all('div', {'class':'border-bottom-0-important'})[0].find('div', {'class':'table-col-1'}).text

'Bautherm Flachdach Spenglerei AG'

In [9]:
#Testen für Lehrstelle
r.find_all('div', {'class':'border-bottom-0-important'})[0].find('div', {'class':'table-col-2'}).text

'Abdichter/in EFZ'

In [10]:
#Testen für Orte
r.find_all('div', {'class':'border-bottom-0-important'})[0].find('div', {'class':'table-col-3'}).text

'Aarburg (AG)'

In [11]:
#Testen für Anzahl Lehrstellen
r.find_all('div', {'class':'border-bottom-0-important'})[0].find('div', {'class':'table-col-5'}).text

'\n1 Lehrstelle(n)\n'

In [12]:
#Zusammenführen
finallist = []

for job in list_of_jobs:
    #print(job.find('div', {'class':'table-col-1'}).text)
    company = job.find('div', {'class':'table-col-1'}).text #company
    j = job.find('div', {'class':'table-col-2'}).text #job
    location = job.find('div', {'class':'table-col-3'}).text #loation

    try:
        count = job.find('div', {'class':'table-col-5'}).text #count
    except:
        count = "N/A"
    minidict = {'company': company,
                'job': j,
                'location': location,
                'count':count}
    finallist.append(minidict)


In [13]:
#DataFrame bauen
pd.DataFrame(finallist)

,company,job,location,count
0,Bautherm Flachdach Spenglerei AG,Abdichter/in EFZ,Aarburg (AG),\n1 Lehrstelle(n)\n
1,Abdichtungsbau Durrer AG,Abdichter/in EFZ,Alpnach Dorf (OW),\n2 Lehrstelle(n)\n
2,Riget Dach AG,Abdichter/in EFZ,Bilten (GL),\n2 Lehrstelle(n)\n
3,VISETTI ISOLAZIONI SAGL,Abdichter/in EFZ,Cadro (TI),\n1 Lehrstelle(n)\n
4,ANTONIO CORTI SA,Abdichter/in EFZ,Caslano (TI),\n2 Lehrstelle(n)\n
5,Schoop + Co. AG,Abdichter/in EFZ,Dättwil AG (AG),\n3 Lehrstelle(n)\n
6,Ackermann AG,Abdichter/in EFZ,Düdingen (FR),\n1 Lehrstelle(n)\n
7,Boretti AG,Abdichter/in EFZ,Erlen (TG),\n1 Lehrstelle(n)\n
8,Etanchéité Camélique SA,Abdichter/in EFZ,Farvagny-le-Grand (FR),\n1 Lehrstelle(n)\n
9,Schwager Bedachungen AG,Abdichter/in EFZ,Fischingen (TG),\n1 Lehrstelle(n)\n


In [15]:
## Alle Lehrstellen aller Firmen
df['JobID']

0      51914.1
1      51919.1
2      15008.2
3      15009.2
4      15010.2
        ...   
633    64011.1
634    64012.1
635    30303.1
636    54218.1
637           
Name: JobID, Length: 638, dtype: object

In [ ]:
all_jobs = []
#Mit den Lehrstellen IDs alle Lehrstellen nach Berufen auslesen
url1 = "https://www.berufsberatung.ch/dyn/show/2930?lang=de&Idx=0&OrderBy=1&Order=0&PostBackOrder=0&postBack=true&CountResult=12&Total_Idx=11&CounterSearch=3&UrlAjaxWebSearch=%2FLenaWeb%2FAjaxWebSearch&prof_="
url2 = "&LocName=&LocId=&Area=10&schoolyear_=2020&schoolyear_=2021&langcode_=de&langcode_=fr&langcode_=it&langcode_=rm&langcode_=en"
for ID in df['JobID']:
    r = requests.get(url1+ID+url2)
    r = BeautifulSoup(r.text, "html.parser")
    list_of_jobs = r.find_all('div', {'class':'border-bottom-0-important'})
    
    finallist = []

    for job in list_of_jobs:
        #print(job.find('div', {'class':'table-col-1'}).text)
        company = job.find('div', {'class':'table-col-1'}).text #company
        j = job.find('div', {'class':'table-col-2'}).text #job
        location = job.find('div', {'class':'table-col-3'}).text #loation

        try:
            count = job.find('div', {'class':'table-col-5'}).text #count
        except:
            count = "N/A"
        minidict = {'company': company,
                'job': j,
                'location': location,
                'count':count}
        finallist.append(minidict)
    
    all_jobs = all_jobs + finallist